In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump

In [2]:
# CSVファイルを読み込む
#df = pd.read_csv('scraped_data_forML.csv', index_col=0)

#df.head(2)

In [3]:
import configparser
from sqlalchemy import create_engine

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['database']['user']
db_password = config['database']['password']
db_host = config['database']['host']
db_name = config['database']['name']

# PostgreSQLデータベースへの接続情報
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')
df = pd.read_sql_table('rental_properties', engine)

In [4]:
drop_cols = ['posting_date', 'removed_date', 'predicted_cost_gb', 'predicted_cost_rf']
df.drop(drop_cols, axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               2774 non-null   int64 
 1   property_type    2774 non-null   object
 2   building_name    2774 non-null   object
 3   address          2774 non-null   object
 4   access_1         2774 non-null   object
 5   access_2         2754 non-null   object
 6   access_3         2682 non-null   object
 7   age              2774 non-null   object
 8   building_floors  2774 non-null   object
 9   room_floor       2774 non-null   object
 10  rent             2774 non-null   object
 11  management_fee   2774 non-null   object
 12  deposit          2774 non-null   object
 13  gratuity         2774 non-null   object
 14  layout           2774 non-null   object
 15  area             2774 non-null   object
 16  url              2774 non-null   object
dtypes: int64(1), object(16)
memory us

# 前処理
1. 実質負担額を作成する
   1. rent +  management_fee +  (deposit / 48)
2. access_1, access_2, access_3から最寄駅ではなくて桜山駅からの時間を取得する
2. 数値型に変換する
   1. age, building_floors, room_floor
   2. area

In [6]:
# 共益費management_feeと敷金(deposit)と礼金(gratuity)が「-」の場合は0とする
df['management_fee'] = df['management_fee'].replace('-', '0円')
df['deposit'] = df['deposit'].replace('-', '0万円')
df['gratuity'] = df['gratuity'].replace('-', '0万円')

# 金額関連の列を数値に変換する関数
def yen_to_float(yen_str):
    return float(yen_str.replace('万円', '')) * 10000

# rent, management_fee, deposit, gratuity を数値に変換
df['rent'] = df['rent'].apply(yen_to_float)
df['management_fee'] = df['management_fee'].str.replace('円', '').astype(float)
df['deposit'] = df['deposit'].apply(yen_to_float)
df['gratuity'] = df['gratuity'].apply(yen_to_float)

# 面積を数値に変換
df['area'] = df['area'].str.replace('m2', '').astype(float)

# 実質負担額を計算（rent + management_fee + (deposit + gratuity) / 48）
df['actual_cost'] = df['rent'] + df['management_fee'] + (df['deposit'] + df['gratuity']) / 48

In [7]:
# 桜山駅からの最短時間を取得する関数
def get_min_time_to_sakurayama(access_str):
    if pd.isnull(access_str):
        return np.nan
    times = [int(s.split('歩')[1].replace('分', '')) for s in access_str.split('/') if '桜山駅' in s]
    if times:
        return min(times)
    else:
        return np.nan

# access_1, access_2, access_3から桜山駅までの最短時間を計算
df['min_time_to_sakurayama'] = df[['access_1', 'access_2', 'access_3']].apply(
    lambda x: min(
        filter(pd.notnull, [get_min_time_to_sakurayama(x['access_1']), get_min_time_to_sakurayama(x['access_2']), get_min_time_to_sakurayama(x['access_3'])]),
        default=np.nan
    ),
    axis=1
)

In [8]:
# '築新'または'新'を'0'年として扱うための処理を追加
df['age'] = df['age'].str.replace('築新', '0').str.replace('新', '0').str.replace('築', '').str.replace('年', '')


# 空の値を0に置き換え
df['age'] = df['age'].replace('', '0')

# 整数型に変換
df['age'] = df['age'].astype(int)

# 建物の階数と部屋の階についても同様に処理
df['building_floors'] = df['building_floors'].str.extract('(\d+)')[0].fillna('0').astype(int)
df['room_floor'] = df['room_floor'].str.extract('(\d+)')[0].fillna('0').astype(int)

# ここでのfillna('0')は、抽出した結果がNaN（該当する数値がない場合）の場合に0を割り当てるために使用しています。

In [9]:
df.head()

,id,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,actual_cost,min_time_to_sakurayama
0,7704,賃貸マンション,フロイデ桜山,愛知県名古屋市昭和区菊園町４,地下鉄桜通線/桜山駅 歩11分,地下鉄鶴舞線/川名駅 歩17分,地下鉄桜通線/瑞穂区役所駅 歩19分,6,3,1,80000.0,4500.0,0.0,80000.0,1LDK,46.84,https://suumo.jp/chintai/jnc_000088573366/?bc=...,86166.666667,11
1,7706,賃貸マンション,Ｅｚサイト,愛知県名古屋市昭和区大和町１,地下鉄桜通線/桜山駅 歩6分,地下鉄鶴舞線/川名駅 歩15分,地下鉄桜通線/御器所駅 歩8分,22,3,3,170000.0,0.0,340000.0,170000.0,3SLDK,97.95,https://suumo.jp/chintai/jnc_000076484377/?bc=...,180625.000000,6
2,7709,賃貸マンション,ミズホフジヒロビルディング,愛知県名古屋市瑞穂区瑞穂通１,地下鉄桜通線/桜山駅 歩4分,地下鉄桜通線/瑞穂区役所駅 歩7分,地下鉄桜通線/瑞穂運動場西駅 歩14分,10,7,3,76000.0,8000.0,0.0,0.0,1LDK,38.61,https://suumo.jp/chintai/jnc_000088243858/?bc=...,84000.000000,4
3,7708,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,8,2,1,53500.0,4000.0,0.0,0.0,1SK,20.13,https://suumo.jp/chintai/jnc_000062206827/?bc=...,57500.000000,3
4,7710,賃貸マンション,ミズホフジヒロビルディング,愛知県名古屋市瑞穂区瑞穂通１,地下鉄桜通線/桜山駅 歩4分,地下鉄桜通線/瑞穂区役所駅 歩7分,地下鉄桜通線/瑞穂運動場西駅 歩14分,10,7,4,77000.0,8000.0,0.0,0.0,1LDK,39.79,https://suumo.jp/chintai/jnc_000088588437/?bc=...,85000.000000,4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2774 non-null   int64  
 1   property_type           2774 non-null   object 
 2   building_name           2774 non-null   object 
 3   address                 2774 non-null   object 
 4   access_1                2774 non-null   object 
 5   access_2                2754 non-null   object 
 6   access_3                2682 non-null   object 
 7   age                     2774 non-null   int64  
 8   building_floors         2774 non-null   int64  
 9   room_floor              2774 non-null   int64  
 10  rent                    2774 non-null   float64
 11  management_fee          2774 non-null   float64
 12  deposit                 2774 non-null   float64
 13  gratuity                2774 non-null   float64
 14  layout                  2774 non-null   

## object型の処理
- エンコーディングする
  - property_type
  - address
  - layout
- dropする
  - building_name
  - access_1
  - access_2
  - access_3
  - url
  - posting_date
  - predictions_rf
  - predictions_gb

In [11]:
# エンコーディングするカラム
categorical_cols = ['property_type', 'address', 'layout']

# One-hotエンコーディング
df = pd.get_dummies(df, columns=categorical_cols)

In [12]:
# 不要なカラムを削除
drop_cols = ['building_name', 'access_1', 'access_2', 'access_3', 'url']
df.drop(drop_cols, axis=1, inplace=True)

In [13]:
df.columns = df.columns.astype(str)

# モデルの訓練

In [14]:
# 特徴量とターゲット変数に分割
X = df.drop('actual_cost', axis=1)  # actual_cost以外のすべてのカラムを特徴量として使用
y = df['actual_cost']  # actual_costをターゲット変数として使用

# データセットの分割（例：訓練データ80%、テストデータ20%）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# モデルの訓練とテスト
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

for name, model in models.items():
    # モデルの訓練
    model.fit(X_train, y_train)
    
    # テストデータで予測
    predictions = model.predict(X_test)
    
    # RMSEの計算
    rmse_score = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"{name}: Test RMSE: {rmse_score:.4f}")
    
    # モデルの保存
    dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')

Random Forest: Test RMSE: 1137.8000
Gradient Boosting: Test RMSE: 846.2549
